In [1]:
def plot_get_limits(D, delta=0.25):
    x1_lim = (np.min(D[:,0])-delta, np.max(D[:,0])+delta)
    x2_lim = (np.min(D[:,1])-delta, np.max(D[:,1])+delta)
    
    return (x1_lim, x2_lim)


In [2]:
def plot_init_axis(ax, title=None):
    
    if not title is None:
        ax.set_title(title, fontsize=16)
    ax.set_xlim(xy_limits[0][0], xy_limits[0][1])
    ax.set_ylim(xy_limits[1][0], xy_limits[1][1])
    ax.set_ylabel('$x_2$', fontsize=14)
    ax.set_xlabel('$x_1$', fontsize=14)

In [3]:
def plot_reg_line(ax, beta, bias, lw=5):
    # Plotting data
    x1 = np.linspace(xy_limits[0][0], xy_limits[0][1]) 
    x2 = - beta[0] / beta[1] * x1 - bias/beta[1]
    # Clipping
    ix = [i>xy_limits[1][0] and i < xy_limits[1][1] for i in x2]
    #ix = [i>x2_lim[0] and i < x2_lim[1] for i in x2]
    reg_line = np.c_[x1[ix], x2[ix]]
    # Plotting
    ax.plot(reg_line[:,0], reg_line[:,1], linewidth=lw, c='#FFFF00')
    return reg_line

In [4]:
def plot_data(ax, X, y, alpha=1, X_test=None, y_test=None):
    """ 2D plot of classification task"""
   
    # Plot the training & test points
    ax.scatter(X[:, 0], X[:, 1], c=y, cmap=class_coloring, edgecolors='k', marker='o', alpha=alpha)
    #ax.set_xlabel("$x_1$")
    #ax.set_ylabel("$x_2$")
    if X_test is not None and y_test is not None:
        ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=class_coloring, edgecolors='k', marker='*', s=150, alpha=alpha)

In [5]:
def plot_synthetic(ax, X, y=None, weight=None, add_bar=False):
    """ 2D plot of classification task"""
    
    a = 0.7

    if y is None:
        ax.scatter(X[:,0], X[:,1], marker='^', c='k', s=weight, alpha=a)
    else:
        ax.scatter(X[:, 0], X[:, 1], c=y, cmap=class_coloring, edgecolors='k', marker='^', s=weight, alpha=a)

    if add_bar:
        ax_c = plt.colorbar(plt.cm.ScalarMappable(cmap=class_coloring))
        ax_c.set_label("BB-Modell estimate of $p(y=1|X)$")
        ax_c.set_ticks([0, .25, .5, .75, 1])
        

In [6]:
def mesh_grid_orig(model, res = 0.01, mode='classification'):
    res = 0.01 # step size in the mesh
   
    xx, yy = np.meshgrid( np.arange(xy_limits[0][0], xy_limits[0][1], res),
                          np.arange(xy_limits[1][0], xy_limits[1][1], res)
                        )   
    if mode == 'classification':
        y_hat = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    elif mode =='regression':
        y_hat = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    else:
        raise(BaseException('The mode {} is not supported.'.format(mode)))
    y_hat = y_hat.reshape(xx.shape)
    return (xx, yy, y_hat)

In [7]:
def mesh_grid_data(res = 0.01):
    
    xx, yy = np.meshgrid( np.arange(xy_limits[0][0], xy_limits[0][1], res),
                          np.arange(xy_limits[1][0], xy_limits[1][1], res)
                        )   
    
    return (xx, yy)

In [8]:
def mesh_grid_prediction(model, res = 0.01, mode='classification'):
   
    data = mesh_grid_data(res)
    
    if mode == 'classification':
        y_hat = clf.predict_proba(np.c_[data[0].ravel(), data[1].ravel()])[:, 1]
    elif mode =='regression':
        y_hat = clf.predict(np.c_[data[0].ravel(), data[1].ravel()])
    else:
        raise(BaseException('The mode {} is not supported.'.format(mode)))
    y_hat = y_hat.reshape(data[0].shape)
    return (data[0], data[1], y_hat)

In [9]:
def plot_decision_surface(ax, clf, data=None, add_bar=False, binarize=False):
    
    if data is None:
        x1, x2, y_hat = mesh_grid_prediction(clf)
    else:
        (x1, x2, y_hat) = data
        
    if binarize:
            y_hat = y_hat >= 0.5
            
    contour = ax.contourf(x1, x2, y_hat, cmap=class_coloring, alpha=.5) # , vmin=0, vmax=1
    
    if add_bar:
        ax_c = plt.gcf().colorbar(contour)
        ax_c.set_label("$P(y=1|X)$")
        ax_c.set_ticks([0, .25, .5, .75, 1])
        

In [10]:
def plot_instance(ax, xi=None):
    """ Plot of the observation to be explained"""
    if xi is None:
        xi = Xi
        
    ax.scatter(xi[0,0], xi[0,1], marker='o', c='#FFFF00', edgecolors='k' ,s=250)           

In [11]:
def plot_wb_model_grid(ax, wb_model, resolution=0.2):
    data = mesh_grid_data(res=resolution)
    data = np.c_[data[0].ravel(), data[1].ravel()]
    y_hat = wb_model.predict(data)

    ax.scatter(data[y_hat>0,0], data[y_hat>0,1], c='r', marker='x')
    ax.scatter(data[y_hat<0,0], data[y_hat<0,1], c='b', marker='x')

In [12]:
def plot_explanation(ax, wb_model):
    dim = len(wb_model.coef_)
    labels = ('x1', 'x2')
    barcolor = np.repeat('r', dim)
    for i, w in enumerate(wb_model.coef_):
        if w < 0:
            barcolor[i]='b'

    y_pos = np.arange(dim)        
    ax.barh(y_pos, wb_model.coef_, align='center', color=barcolor)
    ax.set_xlabel('Coefficient of the interpretable model', fontsize=14)
    ax.set_yticks(y_pos)
    ax.set_yticklabels(labels)